In [41]:
import csv
from datetime import datetime, timezone

csv_list = ["redis.redis.csv"] # ["redis.redis.csv"]
'''
pr_author_login(0), pr_author_name(1), ###pr_create_at###(2), ###pr_merged_at###(3), pr_changed_files(4), [0:5] (5)

pr_LOC(5), pr_reviewers(6),###pr_first_review_at(7)###,pr_review_commits(8),pr_review_comments(9),pr_commits(10), [5:11] (6)

repo_stars(11),###repo_created_at(12)###,repo_external_contribution(13),repo_team_size(14), [11:15] (4)

user_followers(15),###user_created_at(16)###,user_experience, [15:18] (3)

user_repo_experience,user_main_member,user_watched(20),user_succ_rate [18:22] (4)

'''
pr_head = ['pr_author_login','pr_author_name','pr_notice_time','pr_review_time','pr_changed_files',
    'pr_LOC','pr_reviewers','pr_review_commits','pr_review_comments','pr_commits'] # 10

repo_head = ['repo_stars','repo_create_time','repo_external_contribution','repo_team_size'] # 4

user_head = ['user_followers','user_dev_time','user_experience', 'user_repo_experience','user_main_member',
             'user_watched','user_succ_rate'] # 7

csv_head = pr_head + repo_head + user_head
length = len(csv_head)
cur_time = datetime.now(timezone.utc)

with open("combine_data.csv", 'w') as f_write:
    csv_write = csv.writer(f_write)
    csv_head = csv_head
    csv_write.writerow(csv_head)
    
    for file_name in csv_list:
        f_read = open("./csv_files/" + file_name, 'r')
        with f_read:
            reader = csv.reader(f_read)
            count = 0
            for read_row in reader:
                if count == 0:
                    count += 1
                    continue
                write_row = []
                # print(read_row)
                merged_at = datetime.strptime(read_row[3], "%Y-%m-%dT%H:%M:%S%z") 
                review_at =  datetime.strptime(read_row[7] + "Z", "%Y-%m-%d %H:%M:%S%z") 
                                                            #2021-05-18 15:26:18 2021-05-22T08:01:10Z
                created_at = datetime.strptime(read_row[2], "%Y-%m-%dT%H:%M:%S%z")
                repo_created_at = datetime.strptime(read_row[12], "%Y-%m-%dT%H:%M:%S%z") 
                user_created_at = datetime.strptime(read_row[16], "%Y-%m-%dT%H:%M:%S%z") 
                
                #print(merged_at)
                #print(review_at)
                #print(cur_time)
                #print(repo_created_at)
                #print(user_created_at)
                
                
                pr_notice_time = merged_at - review_at
                pr_review_time = review_at - created_at
                repo_create_time = cur_time - repo_created_at
                user_dev_time = cur_time - user_created_at
                
                
                write_row.extend(read_row[0:2]) # pr_author_login, pr_author_name
                write_row.append(pr_notice_time.days) # pr_notice_time
                write_row.append(pr_review_time.days) # pr_review_time
                write_row.extend(read_row[4:7]) # pr_changed_files, pr_LOC, pr_reviewers
                write_row.extend(read_row[8:12]) # pr_review_commits, pr_review_comments, pr_commits, repo_stars
                
                write_row.append(repo_create_time.days)
                write_row.extend(read_row[13:16])
                write_row.append(user_dev_time.days)
                write_row.extend(read_row[17:22])
                csv_write.writerow(write_row)


    